# Assignment 2

You have been engaged by the Salt Lake County Regional Economic Development office to investigate the effects of building a sports stadium on nearby property values. One component of your analysis will be to evaluate the effects of building the Rio Tinto Stadium (now America First Credit Union Field) in Sandy, UT (home of Real Salt Lake). Did it have an effect on property values?

To prepare your analysis, you have been provided the MLS sales data for Sandy, UT as well as several other nearby suburbs in Salt Lake County. You may choose which of these to include in your analysis.

### Environment

Need to include the libraries we will be using in this notebook so Julia knows what functions we will be referencing

In [ ]:
using CSV
using DataFrames
using Plots
using Geodesy
using GraphPlot
using Graphs
using Dates
using GLM
using Statistics

### Data
Reading in and cleaning up the MLS data for Sandy. Converting text columns into dummy variables.

#### Dependent Variable:
*Sold Price*

#### Independent Variables:
* Acres
* Square Footage
* Num Beds
* Num Baths
* Garage Size by Cars
* AC
* Central Heating
* Interest Rate (by sold month)
* Unemployment Rate (by sold month)
* NearStadium [w/in 1 mi of stadium or further than 4 mi]
* After Stadium Built [sold before or after construction completion in 2008]

In [ ]:
# Read in comma-separated data into DataFrame
mls_data = CSV.read("../../Data/SandyData.csv", DataFrame);
draper = CSV.read("../../Data/DraperData.csv", DataFrame);
mls_data = vcat(mls_data, draper);

# Drop extra columns and rows w/ missing values
mls_data = mls_data[(mls_data.Latitude .> 0.0) .& (mls_data.Longitude .< 0.0), 
    [:SoldPrice, :SOLDYRMO, :Acres, :TotSqf, :TotBed, :TotBth, :GaragCap, :Latitude, :Longitude, :AirType, :Heat]];
mls_data = dropmissing(mls_data);

# Create Dummy Variables for AC and Heat
mls_data[!, "AC"] = map(occursin, repeat("Central Air", nrow(mls_data)), mls_data.AirType) * 1;
mls_data[!, "CentralHeating"] = map(occursin, repeat("Central", nrow(mls_data)), mls_data.Heat) * 1;
# Drop old heat/air columns
mls_data = select(mls_data, Not(:AirType));
mls_data = select(mls_data, Not(:Heat));

describe(mls_data)

#### Stadium Properties

Calculating in miles the distance from the America First field and adding to our dataframe as a new column. Also creating a dummy variable from the sold time relative to the construction date in 2008

In [ ]:
# Create transformation for coordinates->UTM (1 point = 1 meter)
utm_utah = UTMfromLLA(12, true, wgs84)

# America First Field
stadium_loc = utm_utah(LLA(40.5829, -111.8934, 0.0))

function distance_from_stadium(lat, lon)
    # Convert to UTM
    house_loc = utm_utah(LLA(lat, lon, 0.0));
    # Calculate distance
    diff = stadium_loc.x - house_loc.x, stadium_loc.y - house_loc.y
    dist = √(sum(diff).^2);
    # Convert to miles from meters
    return dist / 1609.3
end

mls_data[!, "distFromStadium"] = map(distance_from_stadium, mls_data.Latitude, mls_data.Longitude);
mls_data[!, "nearStadium"] = 1 * (mls_data.distFromStadium .<= 1);
mls_data[!, "awayFromStadium"] = 1 * (mls_data.distFromStadium .>= 4);
mls_data = mls_data[mls_data.nearStadium + mls_data.awayFromStadium .> 0, :]

describe(mls_data.distFromStadium)

In [ ]:
# Add dummy variables to track different time periods relative to stadium's construction
mls_data[!, "beforeStadiumAnnouncement"] = 1 * (mls_data.SOLDYRMO .< Date(2006, 8, 12));
mls_data[!, "stadiumUnderConstruction"] = 1 * (mls_data.SOLDYRMO .>= Date(2006, 8, 12)) .& (mls_data.SOLDYRMO .<= Date(2008, 10, 9));
mls_data[!, "stadiumBuilt"] = 1 * (mls_data.SOLDYRMO .>= Date(2008, 10, 9));

#### Housing Market

Joining table with unemployment rate and interest rate by month

In [ ]:
# Join U3 Unemployment rates by sold month
unemployment_data = CSV.read("../../Data/FRED/UNRATE-U3.csv", DataFrame);
mls_data = leftjoin(mls_data, unemployment_data, on = :SOLDYRMO => :DATE);
# Join Federal Funds (Interest) rates by sold month
fedfunds_data = CSV.read("../../Data/FRED/FEDFUNDS.csv", DataFrame);
mls_data = leftjoin(mls_data, fedfunds_data, on = :SOLDYRMO => :DATE);

### Directed Acyclic Graph (DAG)
The DAG shows the relationship between the variables represented in our model. The Interest Rate and Unemployment Rate are serving as observation variables for the wide effects of the 2008 Great Recession, or as mediators for its impact as it happens around the same time as the data being evaluated. The effect we want to hone in on, that of the America First stadium being built in Sandy, is observed in terms of a house's distance from the stadium and when the sell happened relative to the construction date, the other nodes representing more salient factors in determining the house's price. Many of these are also interrelated as the larger the plot of land, the bigger the house and the bigger the house, the more room for bedrooms and bathrooms.

In [ ]:
nodes = ["InterestRate", "MortgageCost", "Acres", "TotSqf", "SoldPrice", "TotBed", "TotBth", "CentralHeating", "AC", "UnemploymentRate", "AfterStadiumBuilt",
            "DistFromStadium", "2008GreatRecession"];
edges = [("InterestRate", "MortgageCost"), ("MortgageCost", "SoldPrice"), ("Acres", "TotSqf"), ("TotSqf", "SoldPrice"), ("Acres", "SoldPrice"),
            ("TotBed", "SoldPrice"), ("TotBth", "SoldPrice"), ("TotSqf", "TotBed"), ("TotSqf", "TotBth"), ("CentralHeating", "SoldPrice"), ("AC", "SoldPrice"),
            ("UnemploymentRate", "SoldPrice"), ("AfterStadiumBuilt", "SoldPrice"), ("DistFromStadium", "SoldPrice"), ("DistFromStadium", "AfterStadiumBuilt"),
            ("AfterStadiumBuilt", "DistFromStadium"), ("InterestRate", "UnemploymentRate"), ("2008GreatRecession", "InterestRate"), ("2008GreatRecession", "UnemploymentRate")];
# Construct Graph
g = SimpleDiGraph();
# Vertices are indexed numerically starting at 1
# get_index will be used to reference vertex based on original node list
add_vertices!(g, length(nodes));
get_index(n) = findfirst(x->x==n, nodes)
for edge in edges
    add_edge!(g, get_index(edge[1]), get_index(edge[2]))
end

gplot(g, nodelabel=nodes, layout=spring_layout)

### Functional Form

*SoldPrice* = &beta;<sub>0</sub> + &beta;<sub>1</sub>*AfterStadiumBuilt* + &beta;<sub>2</sub>*NearStadium* + &beta;<sub>3</sub>*SquareFootage* + &beta;<sub>3</sub>*TotBth* + ... &beta;<sub>*N*</sub>*X*<sub>*N*</sub> ... + &beta;<sub>1</sub>&beta;<sub>2</sub>*NearStadium*\**AfterStadiumBuilt* + &epsilon;

In an attempt to isolate the impact the America First stadium had which acts over time and space, the time of the home sale relative to the stadium's construction date and the distance from the stadium are included, both as singular independent variables and together as an interaction variable. Assuming the houses near the stadium went especially up in price (or down) with a stadium now drawing more traffic to the area, the interaction variable being significant in a linear model would capture this effect. The interest rate and the unemployment rate are included in an attempt to eliminate OVB as the stadium finished construction the same time the housing crash was happening meaning there would be a huge shift in housing prices before and after the stadium regardless of whether there was an impact, these serve to mediate those effects. The rest of the independent variables are the typical factors used to determine housing prices: size and amenities. 

#### Influential Data Points

With the large sample size we are working with, there is a wide enough distribution to where a strong majority of the data points have little influence on the resulting regression. There is, however, one extreme outlier in terms of acreage, or the size of the plot of land that the house is on. One sale exists which lies 55 standard deviations above the average plot size giving it extremely high leverage and making it highly influential. As this seems to exist in an extremely unique situation, we will leave out the point from our model

In [ ]:
mls_data[mls_data.Acres .> 12, :]

In [ ]:
# Show all the sales in terms of price by acres
scatter(mls_data.Acres, mls_data.SoldPrice)
# Fit a model around price by acres
acres_fit = lm(@formula(SoldPrice ~ Acres), mls_data);
# Fit a model around price by acres excluding the outlier
sample = mls_data[mls_data.Acres .< 12, :];
sample_fit = lm(@formula(SoldPrice ~ Acres), sample);
# ! is used to modify the existing plot and adding our models to it
plot!(mls_data.Acres, predict(acres_fit))
plot!(sample.Acres, predict(sample_fit))

## Linear Regression Model

Example of setting up a linear regression model equation to examine estimators

In [ ]:
lm(@formula(SoldPrice ~ TotSqf + Acres + TotBth + UNRATE + nearStadium&stadiumUnderConstruction + nearStadium&stadiumBuilt), mls_data)